# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [20]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [22]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,port saint john's,-31.6229,29.5448,14.09,65,0,2.31,ZA,1722998760
1,1,grytviken,-54.2811,-36.5092,-8.43,67,64,2.56,GS,1722998761
2,2,afaahiti,-17.7500,-149.2833,26.93,86,17,9.03,PF,1722998763
3,3,brookings,44.3114,-96.7984,17.27,89,8,3.48,US,1722998764
4,4,st. john's,47.5649,-52.7093,17.10,89,75,6.69,CA,1722998266


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [31]:
%%capture --no-display

# Configure the map plot
humidity_map = city_data_df.hvplot.points("Lng",
                                          "Lat",
                                          geo = True,
                                          tiles = "OSM",
                                          frame_width = 800,
                                          frame_height = 600,
                                          scale = 2,
                                          color = "City",
                                          size = "Humidity")
                                            
humidity_map
                                        
# Display the map
# YOUR CODE HERE

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [34]:
# Narrow down cities that fit criteria and drop any results with null values
reduce_city_df = city_data_df.loc[(city_data_df["Max Temp"] < 27) & (city_data_df["Max Temp"] > 21) & (city_data_df["Wind Speed"] < 4.5) & (city_data_df["Cloudiness"] == 0)]
drop_city = reduce_city_df.dropna()
drop_city

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
21,21,xiva,41.3783,60.3639,22.99,18,0,4.05,UZ,1722998545
92,92,manicore,-5.8092,-61.3003,24.53,48,0,0.69,BR,1722998863
136,136,changji,44.0167,87.3167,26.44,44,0,1.00,CN,1722998913
196,196,karratha,-20.7377,116.8463,25.38,25,0,2.13,AU,1722998982
224,224,bukan,36.5210,46.2089,25.61,24,0,0.40,IR,1722998625
275,275,al bayda',32.7627,21.7551,21.72,58,0,2.34,LY,1722999074
294,294,almacelles,41.7322,0.4372,24.66,72,0,1.54,ES,1722999095
299,299,tiznit,29.5833,-9.5000,22.27,47,0,0.40,MA,1722999101
330,330,tukrah,32.5341,20.5791,26.22,60,0,2.57,LY,1722999136
368,368,mao,39.8885,4.2658,25.31,83,0,1.03,ES,1722999177


### Step 3: Create a new DataFrame called `hotel_df`.

In [36]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = drop_city[["City", "Country", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
# YOUR CODE HERE

# Display sample data
# YOUR CODE HERE

,City,Country,Lat,Lng,Hotel Name
21,xiva,UZ,41.3783,60.3639,
92,manicore,BR,-5.8092,-61.3003,
136,changji,CN,44.0167,87.3167,
196,karratha,AU,-20.7377,116.8463,
224,bukan,IR,36.5210,46.2089,
275,al bayda',LY,32.7627,21.7551,
294,almacelles,ES,41.7322,0.4372,
299,tiznit,MA,29.5833,-9.5000,
330,tukrah,LY,32.5341,20.5791,
368,mao,ES,39.8885,4.2658,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [39]:
# Set parameters to search for a hotel
radius = 10000
params = {"categories": "accommodation.hotel",
          "apiKey": geoapify_key,
          "limit": 20}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    Lat = ["Lat"]
    Lng = ["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle: {Lng}, {Lat}, {radius}"
    params["bias"] = f"proximity: {Lng}, {Lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params = params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
xiva - nearest hotel: No hotel found
manicore - nearest hotel: No hotel found
changji - nearest hotel: No hotel found
karratha - nearest hotel: No hotel found
bukan - nearest hotel: No hotel found
al bayda' - nearest hotel: No hotel found
almacelles - nearest hotel: No hotel found
tiznit - nearest hotel: No hotel found
tukrah - nearest hotel: No hotel found
mao - nearest hotel: No hotel found
portland - nearest hotel: No hotel found
la maddalena - nearest hotel: No hotel found
elizabethtown - nearest hotel: No hotel found
vitinia - nearest hotel: No hotel found
ha'il - nearest hotel: No hotel found


,City,Country,Lat,Lng,Hotel Name
21,xiva,UZ,41.3783,60.3639,No hotel found
92,manicore,BR,-5.8092,-61.3003,No hotel found
136,changji,CN,44.0167,87.3167,No hotel found
196,karratha,AU,-20.7377,116.8463,No hotel found
224,bukan,IR,36.5210,46.2089,No hotel found
275,al bayda',LY,32.7627,21.7551,No hotel found
294,almacelles,ES,41.7322,0.4372,No hotel found
299,tiznit,MA,29.5833,-9.5000,No hotel found
330,tukrah,LY,32.5341,20.5791,No hotel found
368,mao,ES,39.8885,4.2658,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [41]:
%%capture --no-display

# Configure the map plot
hotel_plot = hotel_df.hvplot.points(
    "Lng", 
    "Lat", 
    geo = True, 
    size = "Humidity",
    scale = 2,
    color = "City",
    alpha = 1,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map
hotel_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Hotel Name,Country)